In [1]:
import pandas as pd

raw_df = pd.read_csv("BRCA_mRNA_normalized.csv").T

In [2]:
clinical_df = pd.read_csv("ClinicalData_BRCA_All_CDE.csv").T
clinical_df.columns = clinical_df.iloc[0]
clinical_df = clinical_df.iloc[pd.RangeIndex(len(clinical_df)).drop(0)]

clinical_df.reset_index(inplace=True)

In [22]:
clinical_df.head(2)

Column1,index,bcr_patient_barcode,bcr_patient_uuid,patient_id,vital_status,days_to_death,days_to_last_followup,additional_studies,additional_surgery_locoregional_procedure,additional_surgery_metastatic_procedure,...,tissue_prospective_collection_indicator,tissue_retrospective_collection_indicator,tissue_source_site,tumor_tissue_site,tumor_tissue_site_other,withdrawn,year_of_dcc_upload,year_of_form_completion,year_of_initial_pathologic_diagnosis,cancer_class
0,Column2,tcga-5l-aat0,86c6f993-327f-4525-9983-29c55625593a,aat0,alive,NaN,1477,NaN,NaN,NaN,...,no,yes,5l,breast,NaN,false,2016,2014,2010,non-TNBC
1,Column3,tcga-5l-aat1,16fc3677-0393-4ed1-ad3f-c8355f056369,aat1,alive,NaN,1471,NaN,NaN,NaN,...,no,yes,5l,breast,NaN,false,2016,2014,2010,non-TNBC


In [ ]:
#indexNames = clinical_df[clinical_df ["breast_carcinoma_estrogen_receptor_status"] == "NaN"].index
#clinical_df.drop(indexNames , inplace=True)

In [4]:
raw_df.columns = raw_df.iloc[0]
raw_df = raw_df.iloc[pd.RangeIndex(len(raw_df)).drop(0)]

raw_df.reset_index(inplace=True)

In [5]:
raw_df.head(2)

HYBRIDIZATION R,index,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,...,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932
0,TCGA-A1-A0SB-01,0,4.74089,1.85069,21.1478,85.8551,0,17.2433,0.827206,0,...,7.45149,14.1453,64.3016,2.25372,32.9284,409.425,94.7092,71.3995,93.5262,0.825593
1,TCGA-A1-A0SD-01,0,5.53891,2.28984,13.7579,164.301,0,7.73122,1.64338,0,...,11.5679,21.5641,79.7483,3.68913,30.8529,321.205,86.5112,81.4754,26.6879,0.412797


In [6]:
#function to get patient id
def patient_id (sample_id):
    return str.lower(sample_id[:-3])

#function to classify as healthy or diseased, get last two characters of sample id  
def h_or_d (sample_id):
    x = sample_id[-2:]
    if x == "11":
        return "Healthy"
    else:
        return "Diseased"

In [7]:
raw_df["h_or_d"] = raw_df.apply(lambda x : h_or_d(x["index"]),axis=1) # Column to store if healthy or dis

In [ ]:
raw_df["h_or_d"]

In [8]:
#function to classify as Triple negative or non-TNBC or NA
def triple_negative (er, pr, her2):
    if er == "negative" and pr == "negative" and her2 == "negative":
        return "TNBC"
    elif er == "positive" or pr == "positive" or her2 == "positive":
        return "non-TNBC"
    else:
        return float ("NaN")

In [9]:
# Column to store if TNBC or non-TNBC or NA
clinical_df["cancer_class"] = clinical_df.apply(lambda x : \
triple_negative(x["breast_carcinoma_estrogen_receptor_status"], x["breast_carcinoma_progesterone_receptor_status"], \
                x["lab_proc_her2_neu_immunohistochemistry_receptor_status"]),axis=1) 

In [ ]:
clinical_df["cancer_class"]

In [10]:
raw_df["patient_id"] = raw_df.apply(lambda x : patient_id(x["index"]),axis=1) # Column to store if healthy or dis

In [11]:
classified_df = pd.merge(raw_df, clinical_df[["bcr_patient_barcode", "cancer_class"]], left_on = "patient_id", right_on = "bcr_patient_barcode", how = "inner")
#merge clinical data to RNAseq data

In [21]:
classified_df.head(2)

,index,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,...,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932,h_or_d,patient_id,bcr_patient_barcode,cancer_class,Class
0,TCGA-A1-A0SB-01,0,4.74089,1.85069,21.1478,85.8551,0,17.2433,0.827206,0,...,409.425,94.7092,71.3995,93.5262,0.825593,Diseased,tcga-a1-a0sb,tcga-a1-a0sb,non-TNBC,non-TNBC
1,TCGA-A1-A0SD-01,0,5.53891,2.28984,13.7579,164.301,0,7.73122,1.64338,0,...,321.205,86.5112,81.4754,26.6879,0.412797,Diseased,tcga-a1-a0sd,tcga-a1-a0sd,non-TNBC,non-TNBC


In [12]:
classified_df ["Class"] = classified_df ["cancer_class"]

In [13]:
classified_df.loc [classified_df["h_or_d"] == "Healthy", "Class"] = "Healthy"

In [17]:
final_classified_df = classified_df.drop(['h_or_d', 'patient_id', "bcr_patient_barcode", "cancer_class"], axis=1)
# drop unneeded columns
final_classified_df

In [23]:
# Move Class to second column
col_name="Class"
second_col = final_classified_df.pop(col_name)
final_classified_df.insert(1, col_name, second_col)
# Rename index as Sample-id
final_classified_df = final_classified_df.rename(columns = {"index":"Sample-id"})

In [25]:
final_classified_df = final_classified_df.dropna() # drop samples with NA values from dataset
final_classified_df.head()

,Sample-id,Class,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,...,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932
0,TCGA-A1-A0SB-01,non-TNBC,0,4.74089,1.85069,21.1478,85.8551,0,17.2433,0.827206,...,7.45149,14.1453,64.3016,2.25372,32.9284,409.425,94.7092,71.3995,93.5262,0.825593
1,TCGA-A1-A0SD-01,non-TNBC,0,5.53891,2.28984,13.7579,164.301,0,7.73122,1.64338,...,11.5679,21.5641,79.7483,3.68913,30.8529,321.205,86.5112,81.4754,26.6879,0.412797
2,TCGA-A1-A0SE-01,non-TNBC,0,4.85265,2.25847,30.8176,260.711,0,11.1664,1.18382,...,18.1733,37.0685,94.6203,7.24577,43.9233,459.133,96.8543,100.889,249.484,0.928793
3,TCGA-A1-A0SF-01,non-TNBC,0,2.73841,1.56838,17.6887,138.96,0,17.7419,0.0919118,...,5.84407,14.2198,48.0106,5.67094,21.5072,349.836,46.6339,24.7378,14.2519,0.309598
4,TCGA-A1-A0SG-01,non-TNBC,0,2.8394,1.47428,14.3066,142.381,0,16.5541,0.693015,...,5.30215,13.0204,55.5405,1.19375,20.7346,205.014,92.2643,32.5047,11.0273,0.361197


In [ ]:
final_classified_df.loc[classified_df.Class == "non-TNBC", "Class"].count()

In [ ]:
final_classified_df.loc[classified_df.Class == "TNBC", "Class"].count()

In [ ]:
final_classified_df.loc[classified_df.Class == "Healthy", "Class"].count()

In [ ]:
final_classified_df.to_csv(path_or_buf = "Preprocessed_RNA_data_normalized.csv", index = False) # write to csv